https://yandex.ru/dev/direct/doc/dg/concepts/overview.html

https://yandex.ru/dev/direct/doc/dg-v4/concepts/About.html

https://yandex.ru/dev/id/doc/dg/oauth/concepts/about.html

https://yandex.ru/dev/id/doc/dg/index.html

https://docs.python.org/3/library/json.html

In [1]:
import requests
import json
import pandas as pd

# from flask import Flask, request, jsonify, redirect
# from urllib.parse import urlencode

In [2]:
# параметры приложения
client_id = '2ea405b379c7474e8bf9e1a3bf31ee2a'
client_secret = '86dcb9f2c8664399b553bc98e44da1df'

# я.директ учетная запись 1
login1 = 'sfedyusnin@yandex.ru'
token1 = 'y0_AQAAAABihNGYAAhQ1wAAAADLKvrhX8L69sCOQii3e7pjpQ0tsAU4Ha4'

# я.директ учетная запись 2
login2 = 'test150822-1@yandex.ru' 
token2 = 'y0_AQAAAABj2FshAAhQ1wAAAADLnEO5KjPw9q9tRUuL9DlWVNgRkdHQDEw'

In [3]:
class YandexDirectEcomru:
    def __init__(self, login, token, sandbox=False):
        self.login = login
        self.token = token
        urls = ['https://api.direct.yandex.ru/v4/json/',
                'https://api.direct.yandex.ru/live/v4/json/',
                'https://api.direct.yandex.com/json/v5/']
        
        sandbox_urls = ['https://api-sandbox.direct.yandex.ru/v4/json/',
                        'https://api-sandbox.direct.yandex.ru/live/v4/json/',
                        'https://api-sandbox.direct.yandex.com/json/v5/']
        
        self.head = {"Authorization": 'Bearer'+' '+self.token,
                     "Accept-Language": "ru",
                     "Client-Login": self.login,
                     "Content-Type": "application/json; charset=utf-8"
                    }
        
        if sandbox is True:
            self.urls = sandbox_urls
        else:
            self.urls = urls

    
    def get_campaigns(self, criteries={}):
        """
        Возвращает параметры кампаний, отвечающих заданным критериям.
        Структура описания критериев:
        https://yandex.ru/dev/direct/doc/ref-v5/campaigns/get.html#input__CampaignsSelectionCriteria
        """
        method = 'campaigns'
        body = {"method": "get",
                "params": {"SelectionCriteria": criteries,
                           "FieldNames": ["Id", "Name"]
                          }
               }
        
        response = requests.post(self.urls[2]+method, headers=self.head, data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text

    def create_new_wordstat_report(self, phrases, regions=None, lim=10):
        """
        Запускает на сервере формирование отчета о статистике поисковых запросов,
        возвращает идентификатор будущего отчета.
        """
        body = {"method": "CreateNewWordstatReport",
                "param": {"Phrases": phrases,
                          "GeoID": regions
                         },
                "locale": "ru",
                "token": self.token
               }
        head = {"Accept-Language": "ru",
                "Content-Length": "204",
                "Content-Type": "application/json; charset=utf-8"}
        
        if len(phrases) <= lim:
            response = requests.post(self.urls[1], headers=head, 
#                                      data=json.dumps(body),
                                     data=json.dumps(body, ensure_ascii=False).encode('utf8')         
                                    )
            if response.status_code == 200:
                return response.json()
            else:
                return response.text
        else:
            print('Превышен лимит фраз в одном запросе')

    def get_wordstat_report_list(self):
        """
        Возвращает список сформированных и формируемых отчетов о статистике поисковых запросов
        """
        body = {"method": "GetWordstatReportList",
                "token": self.token}
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
    
    def get_wordstat_report(self, report_id):
        """
        Возвращает отчет о статистике поисковых запросов
        """
        body = {"method": "GetWordstatReport",
                "param": int(report_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def delete_wordstat_report(self, report_id):
        """
        Удаляет отчет о статистике поисковых запросов
        """
        body = {"method": "DeleteWordstatReport",
                "param": int(report_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def create_new_forecast(self, phrases, regions=None, currency='RUB', auc_bids='No'):
        """
        Запускает на сервере формирование прогноза показов, кликов и затрат.
        Возможные значения currency: RUB, CHF, EUR, KZT, TRY, UAH, USD, BYN.
        """
        body = {"method": "CreateNewForecast",
                "param": {"Phrases": phrases,
                          "Categories": [],
                          "GeoID": regions,
                          "Currency": currency,
                          "AuctionBids": auc_bids
                         },
                "locale": "ru",
                "token": self.token
               }
        
        head = {"Accept-Language": "ru",
                "Content-Length": "204",
                "Content-Type": "application/json; charset=utf-8"}
        
        response = requests.post(self.urls[1], data=json.dumps(body, ensure_ascii=False).encode('utf8'))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
            
    def get_forecast_list(self):
        """
        Возвращает список сформированных и формируемых отчетов о прогнозируемом количестве
        показов и кликов, затратах на кампанию
        """
        body = {"method": "GetForecastList",
                "token": self.token}
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def get_forecast(self, forecast_id):
        """
        Возвращает сформированный прогноз показов, кликов и затрат по его идентификатору
        """
        body = {"method": "GetForecast",
                "param": int(forecast_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text 
      
    def delete_forecast_report(self, forecast_id):
        """
        Удаляет отчет о прогнозируемом количестве показов и кликов, затратах на кампанию
        """
        body = {"method": "DeleteForecastReport",
                "param": int(forecast_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        

In [5]:
direct = YandexDirectEcomru(login=login2, token=token2, sandbox=False)

# print(direct.get_campaigns.__doc__)
print(direct.get_campaigns())

print(direct.get_wordstat_report_list())
print(direct.get_forecast_list())

{'result': {'Campaigns': []}}
{'data': []}
{'data': []}


In [6]:
direct.create_new_wordstat_report(phrases=['автоматизация маркетплейсов', 'аналитик продаж на маркетплейсах', \
                                           'интеграция с маркетплейсами', 'копирование карточек озон -товар',\
                                           'копирование карточки товара озон', 'перенос карточек',\
                                           'программа аналитики маркетплейсов'])

{'data': 401410058}

In [7]:
direct.get_wordstat_report_list()

{'data': [{'ReportID': 401410058, 'StatusReport': 'Done'}]}

In [8]:
direct.get_wordstat_report(report_id=401410058)

{'data': [{'Phrase': 'автоматизация маркетплейсов',
   'SearchedWith': [{'Phrase': 'автоматизация маркетплейсов', 'Shows': 246},
    {'Phrase': 'автоматизация работы +с маркетплейсами', 'Shows': 29},
    {'Phrase': 'автоматизация продаж +на маркетплейсах', 'Shows': 28},
    {'Phrase': 'автоматизация маркетплейсов 1с', 'Shows': 21},
    {'Phrase': 'автоматизация торговли +на маркетплейсах', 'Shows': 8}],
   'SearchedAlso': [{'Phrase': 'tf2 market', 'Shows': 1609},
    {'Phrase': 'тф2 маркет', 'Shows': 828},
    {'Shows': 1990, 'Phrase': 'эвотор маркет'},
    {'Phrase': 'ас маркет', 'Shows': 3146},
    {'Phrase': 'варф маркет', 'Shows': 745},
    {'Shows': 17577, 'Phrase': 'асу это'},
    {'Shows': 1106, 'Phrase': 'кейс маркет'},
    {'Phrase': 'я маркет интернет магазин', 'Shows': 1043},
    {'Shows': 735, 'Phrase': 'qtx market net'},
    {'Phrase': 'ртс маркет электронный магазин', 'Shows': 676},
    {'Phrase': 'си маркет', 'Shows': 1914},
    {'Phrase': 'маркет апп', 'Shows': 1811},
 

In [9]:
# direct.delete_wordstat_report(report_id=7040831)

In [10]:
direct.create_new_forecast(phrases=['автоматизация маркетплейсов', 'аналитик продаж на маркетплейсах', \
                                   'интеграция с маркетплейсами', 'копирование карточек озон -товар',\
                                   'копирование карточки товара озон', 'перенос карточек',\
                                   'программа аналитики маркетплейсов'])

{'data': 401413197}

In [11]:
direct.get_forecast_list()

{'data': [{'ForecastID': 401413197, 'StatusForecast': 'Done'}]}

In [12]:
direct.get_forecast(forecast_id=401413197)

{'data': {'Common': {'Clicks': 28,
   'Min': 4282.83,
   'PremiumClicks': 407,
   'Max': 9617.56,
   'Geo': '0',
   'PremiumMin': 195098.72,
   'FirstPlaceClicks': 31,
   'Shows': 4279},
  'Phrases': [{'Clicks': 3,
    'CTR': 2.8,
    'Min': 81.1,
    'Currency': 'RUB',
    'FirstPlaceCTR': 2.68,
    'IsRubric': 'No',
    'Max': 174.62,
    'PremiumClicks': 28,
    'PremiumMin': 378.94,
    'PremiumMax': 1304.67,
    'PremiumCTR': 11.67,
    'FirstPlaceClicks': 3,
    'Shows': 246,
    'Phrase': 'автоматизация маркетплейсов'},
   {'Phrase': 'аналитик продаж на маркетплейсах',
    'Shows': 1249,
    'FirstPlaceClicks': 7,
    'PremiumCTR': 15.99,
    'PremiumMax': 2515.35,
    'PremiumMin': 531.64,
    'PremiumClicks': 177,
    'Max': 405.25,
    'IsRubric': 'No',
    'FirstPlaceCTR': 1.29,
    'Currency': 'RUB',
    'Min': 240.28,
    'CTR': 1.15,
    'Clicks': 6},
   {'PremiumMax': 1850.99,
    'PremiumCTR': 10.63,
    'FirstPlaceClicks': 11,
    'PremiumMin': 387.26,
    'Phrase': 'и

In [13]:
# direct.delete_forecast_report(forecast_id=7037265)

In [14]:
direct.get_forecast_list()

{'data': [{'StatusForecast': 'Done', 'ForecastID': 401413197}]}